Import **Libraries**

In [1]:
from google.colab import drive
import os
import pandas as pd
import os
import requests
import json
import random
from sqlglot import parse_one

About [Dataset](https://www.kaggle.com/datasets/saeedtqp/persian-books-datasettaset)

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Megachat')
dataset = 'data.csv'
df = pd.read_csv(dataset)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
publish_dates = sorted(df['price'].dropna().unique().tolist())
print("مقادیر یکتای ستون publishDate:")
for date in publish_dates:
    print(date)

مقادیر یکتای ستون publishDate:
0
400
450
500
550
560
580
590
600
650
675
700
720
750
760
770
800
820
840
850
860
900
950
960
975
996
998
1000
1050
1080
1090
1100
1125
1150
1200
1250
1275
1300
1320
1350
1400
1425
1440
1450
1500
1540
1570
1575
1600
1650
1680
1700
1750
1800
1875
1880
1900
1950
2000
2025
2100
2200
2205
2250
2300
2325
2350
2400
2450
2470
2475
2500
2550
2600
2625
2640
2650
2700
2750
2800
2850
2900
2950
2975
3000
3100
3120
3150
3200
3250
3300
3350
3360
3370
3375
3400
3450
3480
3500
3550
3600
3700
3720
3750
3800
3850
3900
3950
3960
4000
4025
4050
4080
4100
4200
4250
4300
4350
4375
4400
4500
4550
4600
4650
4690
4700
4725
4750
4796
4800
4875
4900
4950
5000
5100
5200
5250
5300
5350
5400
5500
5600
5700
5750
5775
5800
5850
5900
5950
6000
6100
6200
6250
6300
6400
6475
6500
6600
6650
6700
6750
6800
6900
6970
7000
7100
7160
7200
7250
7300
7350
7360
7400
7500
7550
7600
7680
7700
7750
7800
7840
7875
7900
7950
8000
8100
8200
8240
8250
8300
8400
8450
8500
8550
8560
8600
8625
8700
8750
876

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15708 entries, 0 to 15707
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   book_id          15708 non-null  int64  
 1   title            15708 non-null  object 
 2   description      15707 non-null  object 
 3   price            15708 non-null  int64  
 4   number_of_page   15708 non-null  int64  
 5   PhysicalPrice    15708 non-null  int64  
 6   publishDate      13285 non-null  object 
 7   rating           10951 non-null  float64
 8   publisher        15708 non-null  object 
 9   coveruri         15708 non-null  object 
 10  number_of_q      15708 non-null  int64  
 11  categories       15708 non-null  object 
 12  number_of_cm     15708 non-null  int64  
 13  author_name      15708 non-null  object 
 14  translator_name  7766 non-null   object 
 15  lang             15708 non-null  object 
dtypes: float64(1), int64(6), object(9)
memory usage: 1.9+ MB


API 🔧

In [5]:
API_URL = "https://openrouter.ai/api/v1/chat/completions"
API_KEY = "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"
MODEL = "deepseek/deepseek-chat-v3-0324:free"

Simple **PCFG**(Probabilistic Context-Free)

In [6]:
PCFG_RULES = {
    "<Query>": [
        ("SELECT <Columns> FROM books <WhereClause>", 0.8),
        ("SELECT <Columns> FROM books", 0.2)
    ],
    "<Columns>": [
        ("*", 0.9),
        ("title", 0.05),
        ("title, author_name", 0.05)
    ],
    "<WhereClause>": [
        ("WHERE <Condition>", 0.9),
        ("WHERE <Condition> AND <Condition>", 0.1)
    ],
    "<Condition>": [
        ("price > <Value>", 0.2),
        ("rating > <Value>", 0.15),
        ("author_name = '<Text>'", 0.2),
        ("categories = '<Text>'", 0.2),
        ("publishDate > '<Date>'", 0.15),
        ("number_of_page > <Value>", 0.1)
    ]
}

def generate_with_pcfg(symbol="<Query>"):
    if symbol not in PCFG_RULES:
        return symbol
    production = random.choices(PCFG_RULES[symbol], weights=[p[1] for p in PCFG_RULES[symbol]])[0][0]
    return " ".join(generate_with_pcfg(sym) for sym in production.split())

Calling LLM --> generate with PCFG

In [7]:
def call_llm(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }
    try:
        response = requests.post(API_URL, headers=headers, json=data)
        response.raise_for_status()
        response_json = response.json()
        if "choices" in response_json and len(response_json["choices"]) > 0:
            return response_json["choices"][0]["message"]["content"].strip()
        else:
            return generate_with_pcfg()
    except requests.exceptions.RequestException as e:
        print(f"خطا در درخواست به API: {e}")
        return generate_with_pcfg()

Query modification using AST(AST-based Retriever)

In [8]:
def refine_with_ast(sql_query, reference_sqls):
    try:
        parsed_query = parse_one(sql_query)
        best_match = None
        best_similarity = 0
        for ref_sql in reference_sqls:
            parsed_ref = parse_one(ref_sql)
            similarity = parsed_query.similarity(parsed_ref)
            if similarity > best_similarity:
                best_similarity = similarity
                best_match = ref_sql
        return best_match if best_similarity > 0.7 else sql_query
    except Exception as e:
        print(f"خطا در پردازش AST: {e}")
        return sql_query

LLM, generate SQL queries

In [9]:
def generate_sql_query(user_question):
    prompt = f"""
تو متخصص تولید کوئری SQL از زبان فارسی هستی.
Schema پایگاه داده من:
جدول: books
ستون‌ها:
- book_id (integer): شناسه کتاب
- title (text): عنوان کتاب
- description (text): توضیحات
- price (integer): قیمت دیجیتال (تومان)
- number_of_page (integer): تعداد صفحات
- PhysicalPrice (integer): قیمت فیزیکی (تومان)
- publishDate (text, format: YYYY/MM/DD): تاریخ انتشار
- rating (float): امتیاز
- publisher (text): ناشر
- coveruri (text): آدرس تصویر جلد
- number_of_q (integer): تعداد سؤالات
- categories (text): دسته‌بندی
- number_of_cm (integer): تعداد نظرات
- author_name (text): نام نویسنده
- translator_name (text): نام مترجم
- lang (text): زبان

سؤال کاربر: "{user_question}"
فقط کوئری SQL رو برگردان، بدون توضیح اضافه ای.
"""
    sql_query = call_llm(prompt)
    if not sql_query:
        sql_query = generate_with_pcfg()
    return refine_with_ast(sql_query, ["SELECT * FROM books WHERE price > 5000", "SELECT * FROM books WHERE rating > 4"])


Run system

In [11]:
def main():
    while True:
        question = input("سؤالت از من چیه؟ (اگه کارت با من تموم شده بنویس 'بوس'): ")
        if question.lower() == "بوس":
            break
        sql_query = generate_sql_query(question)
        print(f"کوئری: {sql_query}")

if __name__ == "__main__":
    main()

سؤالت از من چیه؟ (اگه کارت با من تموم شده بنویس 'بوس'): کتاب هایی که قیمتشون بیشتر از 8000 هستش
خطا در پردازش AST: Invalid expression / Unexpected token. Line 1, Col: 3.
  ```sql
SELECT * FROM books WHERE price > 8000;
```
کوئری: ```sql
SELECT * FROM books WHERE price > 8000;
```
سؤالت از من چیه؟ (اگه کارت با من تموم شده بنویس 'بوس'): کتاب هایی که قیمتشون بیشتر از 8000 یا کمتر از 10000
خطا در پردازش AST: Invalid expression / Unexpected token. Line 1, Col: 3.
  ```sql
SELECT * 
FROM books 
WHERE price > 8000 OR price < 10000;
```
کوئری: ```sql
SELECT * 
FROM books 
WHERE price > 8000 OR price < 10000;
```
سؤالت از من چیه؟ (اگه کارت با من تموم شده بنویس 'بوس'): کتاب هایی که قیمتشون 8000 هستش و موضوعشون فیزیک هستش
خطا در پردازش AST: Invalid expression / Unexpected token. Line 1, Col: 3.
  ```sql
SELECT * 
FROM books 
WHERE price = 8000 AND categories LIKE '%فیزیک%';
```
کوئری: ```sql
SELECT * 
FROM books 
WHERE price = 8000 AND categories LIKE '%فیزیک%';
```
سؤالت از من چیه؟ (اگه کارت با 